# Evaluating Vectara With RAGAs

In [1]:
#
# On Mac:
# brew install libmagic
#
# Additional dependecies
# pip install ragas pandas python-magic datasets langchain langchainhub langchain-experimental
# pip install -U ragas (version 0.1.5 or higher)
#

In [2]:
from datasets import Dataset
import os
import re
import shutil
import requests
import magic
import numpy as np
import time

from langchain import hub
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    answer_correctness,
    answer_similarity
)
from datasets import Dataset

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

from vectaraClient import VectaraClient

In [3]:
customer_id = "<YOUR-VECTARA-CUSTOMER-ID>"
corpus_id = "<YOUR-VECTARA-CORPUS-ID>"
api_key = "<YOUR-VECTARA-API-KEY>"

## Part 1: Generate Synthetic Evaluation Dataset from a Vectara Corpus

For RAGAs to generate synthetic data, we will utilize OpenAI embedding and Chat. Make sure to have your OPENAI_API_KEY available in the environment. First we use the VectaraClient to download URLs of each document in our corpus (`url` in this case is a metadata field) and use those to download the actual content to a local `temp` folder:

In [ ]:
vc = VectaraClient(api_key, customer_id, corpus_id)

urls = vc.get_all_doc_urls()
print(f"Found {len(urls)} documents in corpus")

In [ ]:
def add_correct_ext(file_path):
    mime = magic.Magic(mime=True)
    mime_type = mime.from_file(file_path)
    
    # Dictionary mapping some MIME types to file extensions
    extension_mapping = {
        'application/pdf': '.pdf',
        'text/html': '.html',
        'text/plain': '.txt',
        'text/markdown': '.md',
        'application/vnd.ms-powerpoint': '.ppt',
        'application/msword': '.doc',
    }
    
    extension = extension_mapping.get(mime_type, '')    
    if extension:
        new_file_path = f"{file_path}{extension}"
        os.rename(file_path, new_file_path)
        return new_file_path
    else:
        print(f"Unsupported file type or no extension mapping found: mime_type={mime_type}.")
        return file_path

In [ ]:
data_dir = './temp'
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.makedirs(data_dir, exist_ok=True)

for url in urls:
    file_name = url.split('/')[-1]
    if len(file_name)==0:
        file_name = 'root'
    file_path = os.path.join(data_dir, file_name)
    
    response = requests.get(url)
        
    # Check if the download was successful
    if response.status_code == 200:
        # Write the content to a file in the specified directory
        with open(file_path, 'wb') as file:
            file.write(response.content)
        file_path = add_correct_ext(file_path)
    else:
        print(f"Failed to download the file (url={url}). Status code: {response.status_code}")

Now that all files are available locally, we load them as LangChain documents and use the RAGAs `TestsetGenerator` functionality to generate 50 synthetic question/answer pairs:

In [ ]:
n_questions = 50

loader = DirectoryLoader(data_dir, use_multithreading=True, silent_errors=True)
documents = loader.load()
for document in documents:
    document.metadata['file_name'] = document.metadata['source']

In [ ]:
gen_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4-turbo-preview")
emb = OpenAIEmbeddings(model="text-embedding-3-large")

generator = TestsetGenerator.from_langchain(generator_llm=gen_llm, 
                                            critic_llm=critic_llm,
                                            embeddings=emb)
testset = generator.generate_with_langchain_docs(documents, test_size=n_questions, 
                                                 raise_exceptions=False, with_debugging_logs=False, is_async=False,
                                                 distributions={simple: 0.5, reasoning: 0.2, multi_context: 0.3})

In [ ]:
testset_df = testset.to_pandas()
testset_df = testset_df[pd.isnull(testset_df.ground_truth)==False]
testset_df = testset_df[['question', 'ground_truth', 'contexts']].dropna()

In [5]:
testset_df.head(2)

question  \
0                                                                  What are some examples of use cases for the Vectara platform?   
1  How can developers customize prompts with metadata using Vectara's Custom Retrieval Augmented Generation (RAG) Prompt Engine?   

                                                                                                                                                                                                                                                                                                                                                                                                                          ground_truth  \
0  The Vectara platform has a unique ability to understand and process information, using hybrid search to find the most relevant products, support cases, and documents that answer user's questions first. It can power chatbots, Q&A systems, conversational applications, and websites based on relevant information. Vectara also provides result recommendations and enables global collaboration through cross-language search.   
1                                                                                                                                                                   Vectara empowers developers with a flexible way of customizing prompts with metadata through the Custom Retrieval Augmented Generation (RAG) Prompt Engine. Developers can use available prompt variables and functions to customize prompts based on their needs.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## Evaluate your Vectara RAG

In [ ]:
sum_top_k = 5

In [6]:
def get_response(row):
    query = row['question']
    response, contexts = vc.query(query, cfg)
    response = re.sub("\[\d+(,\s*\d+)*\]", "", str(response)).replace(' .', '.')
    return pd.Series([response, contexts], index=['answer', 'contexts'])

In [ ]:
# Here we also add a calculation of HHEM - Vectara's Hallucination Detection Model
# HHEM is a factual consistency score, similar to faithfulness, 
# ...but computed directly as a model instead of using LLM-as-a-judge as in RAGAs

from sentence_transformers import CrossEncoder
def calc_hhem(contexts, answer):
    model = CrossEncoder('vectara/hallucination_evaluation_model')
    scores = model.predict([[context, answer] for context in contexts])
    return np.mean(scores)


In [7]:

def eval_rag(df):
    df2 = df.copy()
    df2[['answer', 'contexts']] = df2.apply(get_response, axis=1)
    result = evaluate(
        Dataset.from_pandas(df2),
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness
        ],
        llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0),
        raise_exceptions=False
    )    

    result['hhem_score'] = np.mean(np.vectorize(calc_hhem)(df2['contexts'], df2['answer']))
    return result

In [8]:
cfg = {
        'lambda': 0.0,
        'max_summary_result': sum_top_k,
        'mmr': True,
        'prompt_name': 'vectara-experimental-summary-ext-2023-10-23-small'
    }

latency = []
res = eval_rag(testset_df)
res

Evaluating:   0%|          | 0/184 [00:00<?, ?it/s]

{'faithfulness': 0.9507, 'answer_relevancy': 0.9453, 'answer_similarity': 0.9456, 'answer_correctness': 0.5154, 'hhem_score': 0.7285}

In [9]:
cfg = {
        'lambda': 0.0,
        'max_summary_result': sum_top_k,
        'mmr': False,
        'prompt_name': 'vectara-experimental-summary-ext-2023-10-23-small'
    }

latency = []
res = eval_rag(testset_df)
res

Evaluating:   0%|          | 0/184 [00:00<?, ?it/s]

{'faithfulness': 0.9561, 'answer_relevancy': 0.9395, 'answer_similarity': 0.9407, 'answer_correctness': 0.5260, 'hhem_score': 0.7426}

In [10]:
cfg = {
        'lambda': 0.025,
        'max_summary_result': sum_top_k,
        'mmr': False,
        'prompt_name': 'vectara-experimental-summary-ext-2023-10-23-small'
    }

latency = []
res = eval_rag(testset_df)
res

Evaluating:   0%|          | 0/184 [00:00<?, ?it/s]

{'faithfulness': 0.9409, 'answer_relevancy': 0.9490, 'answer_similarity': 0.9490, 'answer_correctness': 0.5580, 'hhem_score': 0.7113}

In [11]:
cfg = {
        'lambda': 0.025,
        'max_summary_result': sum_top_k,
        'mmr': False,
        'prompt_name': 'vectara-experimental-summary-ext-2023-12-11-large'
    }

latency = []
res = eval_rag(testset_df)
res

Evaluating:   0%|          | 0/184 [00:00<?, ?it/s]

{'faithfulness': 0.9896, 'answer_relevancy': 0.9391, 'answer_similarity': 0.9530, 'answer_correctness': 0.5953, 'hhem_score': 0.6177}